In [35]:
# 2. What's Cooking
# (b)

import pandas as pd
import numpy as np
from sklearn import cross_validation, naive_bayes, linear_model

df_train = pd.read_json("/Desktop/train.json")
df_test = pd.read_json("/Desktop/test.json")
print("Number of dishes: " + str(df_train.shape[0]))

cuisine = df_train.cuisine.unique()
print("Types of cuisine: " + str(cuisine.shape[0]))

ing = []
for i in range(39774):
    ing += df_train.ingredients.iloc[i]
ing_unique = np.unique(np.array(ing))
print("Number of unique ingredients: " + str(ing_unique.size))

Number of dishes: 39774
Types of cuisine: 20
Number of unique ingredients: 6714


In [24]:
# (c)

a = []
for i in range(39774):
    b = np.zeros(6714)
    for g in df_train.ingredients.iloc[i]:
        index = np.where(ing_unique == g)[0][0]
        b[index] = 1
    a.append(b)
X_train = np.array(a)
X_train.shape

(39774, 6714)

In [26]:
a = []
for i in range(df_test.shape[0]):
    b = np.zeros(6714)
    for g in df_test.ingredients.iloc[i]:
        index = np.where(ing_unique == g)[0]
        b[index] = 1
    a.append(b)
X_test = np.array(a)
X_test.shape

(9944, 6714)

In [38]:
# (d)

Y_train = df_train.cuisine.values
scores_gaussian, scores_bernoulli = [], []
for train, test in cross_validation.KFold(len(Y_train), n_folds = 3, shuffle = True):
    print("New fold")
    print("Train on", train)
    print("Test on", test)
    cls_gaussian = naive_bayes.GaussianNB()
    score_gaussian = cls_gaussian.fit(X_train[train], Y_train[train]).score(X_train[test], Y_train[test])
    cls_bernoulli = naive_bayes.BernoulliNB()
    score_bernoulli = cls_bernoulli.fit(X_train[train], Y_train[train]).score(X_train[test], Y_train[test])
    scores_gaussian.append(score_gaussian)
    scores_bernoulli.append(score_bernoulli)
    print(score_gaussian, score_bernoulli)
print("Average accuracy using Gaussian prior is:", np.mean(np.array(scores_gaussian)))
print("Average accuracy using Bernoulli prior is:", np.mean(np.array(scores_bernoulli)))

New fold
Train on [    0     3     4 ..., 39771 39772 39773]
Test on [    1     2     9 ..., 39761 39762 39770]
0.36815507618 0.690073917635
New fold
Train on [    0     1     2 ..., 39771 39772 39773]
Test on [    3     6     8 ..., 39767 39768 39769]
0.376904510484 0.680268517122
New fold
Train on [    1     2     3 ..., 39768 39769 39770]
Test on [    0     4     5 ..., 39771 39772 39773]
0.397043294615 0.682455875698
Average accuracy using Gaussian prior is: 0.380700960426
Average accuracy using Bernoulli prior is: 0.684266103485


In [39]:
# (f)

scores_logistic = []
for train, test in cross_validation.KFold(len(Y_train), n_folds = 3, shuffle = True):
    print("New fold")
    print("Train on", train)
    print("Test on", test)
    cls_logistic = linear_model.LogisticRegression()
    score_logistic = cls_logistic.fit(X_train[train], Y_train[train]).score(X_train[test], Y_train[test])
    scores_logistic.append(score_logistic)
    print(score_logistic)
print("Average accuracy using logistic regression is: ", np.mean(np.array(scores_logistic)))

New fold
Train on [    1     3     6 ..., 39771 39772 39773]
Test on [    0     2     4 ..., 39766 39768 39770]
0.777869965304
New fold
Train on [    0     1     2 ..., 39771 39772 39773]
Test on [    3     6     7 ..., 39756 39762 39764]
0.773872378941
New fold
Train on [    0     2     3 ..., 39766 39768 39770]
Test on [    1    10    19 ..., 39771 39772 39773]
0.773344395836
Average accuracy using logistic regression is:  0.77502891336


In [43]:
# (g)

cls = linear_model.LogisticRegression()
predict = cls.fit(X_train, Y_train).predict(X_test)
final = pd.Series(predict)
result = pd.DataFrame(df_test.id)
result["cuisine"] = final
result.to_csv("/Desktop/cooking.csv", index = False)